In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np
import numpy.linalg as la
from scipy.integrate import RK45

import sympy as sym

from table_maker import *
from functools import partial
from itertools import *
from math import ceil

import pickle

def cos_bell(x, center=0, width=2*np.pi, height=1):
    return (np.cos((x-center)/width*2*np.pi)+1)/2*height * np.heaviside(x-center+width/2,0) * np.heaviside(-x+center+width/2,0)

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size': 24})

From [Kilpatrick and Bressloff 2010](https://doi.org/10.1016/j.physd.2009.06.003)
$$\begin{align*}
    \mu u_t &= -u + \int_{-\infty}^\infty w(x,x^\prime) q(x^\prime,t) f( u(x^\prime,t) - a(x^\prime,t)) \ dx^\prime \\
    q_t &= \frac{1 - q}{\alpha} - \beta q f(u - a) \\
    \epsilon a_t &= -a + \gamma f(u - a)
\end{align*}$$

Modified version - remove synaptic depression $q$
$$\begin{align*}
    \mu u_t &= -u + \int_{-\infty}^\infty w(x,x^\prime) f( u(x^\prime,t) - a(x^\prime,t)) \ dx^\prime \\
    \alpha a_t &= -a + \gamma f(u - a)
\end{align*}$$
(note that parameters have been relabeled)

# Traveling Pulse

In [47]:
###############################################
# Parameters ##################################

θ = .1
α = 5
γ = 1
μ = 1

def firing_rate(u):
    return np.heaviside(u-θ, .5)

def w(x,y):
    return .5*np.exp( - np.abs( np.subtract.outer(x, y) ) )

###############################################
# Simulation parameters #######################

# space
a, b = -50, 150
n = 10**3 * 1
xs = np.linspace(a,b,n)

# time
t0 = 0
t_final = 25
k = 1e-2

# initial conditions

# u0 = np.zeros((2,n))
# u0[0] = cos_bell(xs, center=0, width=50)

with open('profile.pickle', 'rb') as f:
    _, _, _, param_list = pickle.load(f)
    param_keys = ['μ', 'α',  'γ',  'θ', 'Δ', 'c']
    params = {key:val for key,val in zip(param_keys, param_list)}
    
def Unum(ξ, μ, α, γ, θ, Δ, c):
    print(ξ.shape)
    return (1.0/2.0)*(1 - np.exp(-Δ))*np.exp(-ξ)*(lambda input: np.heaviside(input,0.5))(ξ)/(μ*c + 1) + ((1.0/2.0)*(-np.exp(Δ)/(μ*c - 1) + 1.0/(μ*c - 1))*np.exp(ξ) + (( lambda base, exponent: base**exponent )(μ, 2)*( lambda base, exponent: base**exponent )(c, 2)*np.exp(Δ/(μ*c)) - ( lambda base, exponent: base**exponent )(μ, 2)*( lambda base, exponent: base**exponent )(c, 2) - 1.0/2.0*μ*c + θ*(( lambda base, exponent: base**exponent )(μ, 2)*( lambda base, exponent: base**exponent )(c, 2) - 1) + (1.0/2.0)*(μ*c - 1)*np.exp(-Δ) + 1.0/2.0)*np.exp(ξ/(μ*c))/(( lambda base, exponent: base**exponent )(μ, 2)*( lambda base, exponent: base**exponent )(c, 2) - 1))*(lambda input: np.heaviside(input,0.5))(-Δ - ξ) + ((θ + (-( lambda base, exponent: base**exponent )(μ, 2)*( lambda base, exponent: base**exponent )(c, 2) - 1.0/2.0*μ*c + ((1.0/2.0)*μ*c - 1.0/2.0)*np.exp(-Δ) + 1.0/2.0)/(( lambda base, exponent: base**exponent )(μ, 2)*( lambda base, exponent: base**exponent )(c, 2) - 1))*np.exp(ξ/(μ*c)) + 1 - 1.0/2.0*np.exp(-Δ)*np.exp(-ξ)/(μ*c + 1) + (1.0/2.0)*np.exp(ξ)/(μ*c - 1))*(lambda input: np.heaviside(input,0.5))(-ξ)*(lambda input: np.heaviside(input,0.5))(Δ + ξ)

def Anum(ξ, μ, α, γ, θ, Δ, c):
    return γ*(1 - np.exp(ξ/(α*c)))*(lambda input: np.heaviside(input,0.5))(-ξ)*(lambda input: np.heaviside(input,0.5))(Δ + ξ) + γ*(np.exp(Δ/(α*c)) - 1)*np.exp(ξ/(α*c))*(lambda input: np.heaviside(input,0.5))(-Δ - ξ)

u0 = np.zeros((2,n))
u0[0] = Unum(xs, **params)
u0[1] = Anum(xs, **params)



############################################### DO NOT EDIT BELOW ###############################################

# Time integrator

def RK4_step(F, t, u, dt):
    k1 = F(t,u)
    k2 = F(t+dt/2, u + dt/2*k1)
    k3 = F(t+dt/2, u + dt/2*k2)
    k4 = F(t+dt, u + dt*k3)
    return u + dt/6*(k1+2*k2+2*k3+k4)

def F(t,u):
    temp = firing_rate(u[0] - u[1])
    ret = np.array([
        1/μ * (-u[0] + M@(temp)),
        (-u[1] + γ*temp)/α
    ])
    return ret

###############################################
# Simulation ##################################

h = xs[1]-xs[0]
M = w(xs, xs) * h



########################
steps = int(np.ceil( (t_final - t0)/k ))
k = (t_final - t0)/steps

us = [u0]
ts = [t0]

stimulus = np.ones((2,n)) * 0.05 / μ
stimulus[1]*= 0
t_stim = 5

for step in range(steps):
    ts += [ts[-1] + k]
    us += [ RK4_step(F, ts[-1], us[-1], k) ]
    if abs(ts[-1] - t_stim) < k/2:
        print('test')
        us[-1] += stimulus
    print('step %d/%d' % (step,steps), end='\r')
    
js = [ us[i][0]-us[i][1] for i in range(steps+1) ]

(1000,)
test 498/2500


In [48]:
# # export profile

# import pickle
# import sympy as sym
# # approximate pulse-width and speed
# width = ( np.sum(js[-1]>=θ) - 1 )*h # number of intervals above θ
# speed_t0, speed_tf = 20, 40
# speed_t0_index, speed_tf_index = [int(t/k) for t in [speed_t0, speed_tf]]
# pulse_start = np.max(xs[us[speed_t0_index][0]>θ])
# pulse_stop = np.max(xs[us[speed_tf_index][0]>θ])
# speed = (pulse_stop - pulse_start)/(ts[speed_tf_index] - ts[speed_t0_index])
# print('Width: %g' % width)
# print('Speed: %g' % speed)

# param_list = [μ, α,  γ,  θ, width, speed]

# # profile
# threshold_index1 = np.argmin( np.abs(us[-1][0] - θ))
# threshold_index2 = np.argmin( np.abs(us[-1][0][threshold_index1+5:] - θ)) + threshold_index1+5
# my_slice = slice(int(threshold_index1*.9),int(threshold_index2*1.1))
# xis = xs[my_slice]-xs[threshold_index2]
# plt.plot(xis, us[-1][0][my_slice])
# plt.plot(xis, us[-1][1][my_slice])
# with open('profile.pickle', 'wb') as f:
#     pickle.dump([xis, us[-1][0][my_slice], us[-1][1][my_slice], param_list], f)

In [49]:
###############################################
# Animation ###################################



stride = 20

fig, axes = plt.subplots(2, 1, figsize=(12,12))
line_u, = axes[0].plot(xs, us[0][0], 'b-', label="$u$")
line_a, = axes[0].plot(xs, us[0][1], 'm-', label="$a$")


line_j, = axes[1].plot(xs, js[0], 'g-', label="$j=u-a$")
axes[1].plot(xs, θ + 0*xs, 'k--', label='$\\theta$')


y_min = np.min(us)
y_max = np.max(us)
y_min -= .05*np.abs(y_max - y_min)
y_max += .05*np.abs(y_max - y_min)
window = y_min, y_max
axes[0].set_ylim(*window)

y_min = np.min(js)
y_max = np.max(js)
y_min -= .05*np.abs(y_max - y_min)
y_max += .05*np.abs(y_max - y_min)
window = y_min, y_max
axes[1].set_ylim(*window)

for ax in axes:
    ax.legend(loc='right')

#     ax.set_xlim(-50, b)
    ax.set_xlim(a,b)

Δ = -np.log(1 - 4*θ)
plt.plot([150, 150+Δ], [θ]*2, 'k--')

def animate(i):
    print('step %d/%d' % (i,len(ts)), end='\r')
    line_u.set_ydata(us[i][0])
    line_a.set_ydata(us[i][1])
    line_j.set_ydata(js[i])
    return line_u,


# Init only required for blitting to give a clean slate.
def init():
    line_u.set_ydata(us[0])
    return line_u,

anim = animation.FuncAnimation(fig, animate, np.arange(0,len(ts),stride), init_func=init,
                              interval=1/24*1000, blit=True)

plt.close()
HTML(anim.to_html5_video())